In [1]:
# NO NEED!!! IT HAS NOT ACCOMODATED FOR AUTOMATIC DATA FIELD INSERTION SUCH AS "PD", "GS", AND "COST" WITH 3 COLUMNS, SOLUTION: IT CAN BE ACCOMPLISHED BY MANUALLY COPY-PASTING TO EXCEL, NEED ONLY A BIT OF TIME 
# DONE!!!. CHECK THE MANUALLY IMPORTED DATA FOR THOSE 3 FIELDS AND THE ONES WHICH IMPORTED BY POWERMODELS, THE NUMBERS are now THE SAME. the original .m file is 100x larger (pd) and 100x smaller (cost) than the imported powermodels done

In [1]:
# DON'T DELETE
using PowerModels
using Ipopt
using JuMP
using Clp
using Pkg
using Distributions
using MathOptInterface
using ProgressMeter
using DataFrames, XLSX
using LinearAlgebra
# using MathProgBase
using JSON
using Random

In [2]:
# DON'T DELETE THIS CELL IS DEDICATED FOR EXPORTING THE RESULT OF PowerModels.build_ref INTO JSON FILE
# for i in keys(data_stringified)
#     open("$i.json", "w") do file
#     JSON.print(file, data_stringified[i])
#     end
# end
# data_stringified = Dict(string(k) => v for (k, v) in my_dict)
# open("all.json", "w") do file
#     JSON.print(file, data_stringified)
# end
# files = readdir()
# # m_files = filter(x -> endswith(x, ".m"), files);
# my_dict = Dict{String, Any}()
# for i in m_files
#     my_dict[i] = PowerModels.build_ref(PowerModels.parse_file(i))[:it][:pm][:nw][0]
# end

In [2]:
# DONT' DELETE
struct Bus
    pd::Float64
    gs::Float64
    gens::Vector{Int}
end

struct Gen
    bus::Int
    pmin::Float64
    pmax::Float64
    pstart::Float64
    cost::Vector{Float64}
end

struct Line
    rate::Float64
    frombus::Int
    tobus::Int
    β::Float64
end

mutable struct NetworkReference
    ref::Dict{Symbol,Any}
    nbus::Int
    ngen::Int
    nline::Int
    r::Int # reference bus
    bus::Vector{Bus}
    gen::Vector{Gen}
    line::Vector{Line}
    originalindices::Dict{Symbol,Vector{Int}}
    B::Matrix{Float64}
    π::Matrix{Float64}
    stdω::Vector{Float64}
    line_prob::Float64
    bus_prob::Float64
end

mutable struct SingleScenarioOPF
    model::JuMP.Model
    p::Vector{JuMP.VariableRef}  # Store variable references
    ω::Vector{JuMP.VariableRef}  # Store variable references
end

# 👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆👇👆

function NetworkReference(ref::Dict{Symbol,Any};
        line_prob::Float64 = 0.9, bus_prob::Float64 = 0.9,
        σscaling::Float64 = 0.05
    )
    function generateindices(d::Dict)
        originalindices = sort(collect(keys(d))); nindices = length(originalindices)
        reverseindices = Dict(zip(originalindices,1:nindices))
        nindices, originalindices, reverseindices
    end
    ngen, genindices, gen_index = generateindices(ref[:gen])
    gen = [Gen(
        ref[:gen][genindices[i]]["gen_bus"],
        ref[:gen][genindices[i]]["pmin"],
        ref[:gen][genindices[i]]["pmax"],
        getstart(ref[:gen],genindices[i],"pg_start"),
        ref[:gen][genindices[i]]["cost"]
    ) for i in 1:ngen]
    nbus, busindices, bus_index = generateindices(ref[:bus])
    bus = [Bus(
        ref[:bus][busindices[i]]["pd"],
        ref[:bus][busindices[i]]["gs"],
        [gen_index[g] for g in ref[:bus_gens][busindices[i]]]
    ) for i in 1:nbus]
    nline, lineindices, line_index = generateindices(ref[:branch])
    line = [Line(
        ref[:branch][lineindices[l]]["rate_a"],
        bus_index[ref[:branch][lineindices[l]]["f_bus"]],
        bus_index[ref[:branch][lineindices[l]]["t_bus"]],
        1 / ref[:branch][lineindices[l]]["br_x"]
    ) for l in 1:nline]
    originalindices = Dict(:bus => busindices, :gen => genindices, :line => lineindices)

    length(ref[:ref_buses]) > 1 && warn("Using only the first reference bus")
    r = bus_index[ref[:ref_buses][first(keys(ref[:ref_buses]))]["bus_i"]]

    nonref_indices = [b for b in 1:nbus if b != r]
    B = admittancematrix(ref, bus_index)
    π = zeros(nbus,nbus)
    π[nonref_indices, nonref_indices] = inv(B[nonref_indices, nonref_indices])

    stdω = [σscaling*ref[:bus][busindices[i]]["pd"] for i in 1:nbus]

    NetworkReference(ref, nbus, ngen, nline, r, bus, gen, line,
        originalindices, B, π, stdω, line_prob, bus_prob)
end

NetworkReference(filename::String; kwargs...) = NetworkReference(
    PowerModels.build_ref(PowerModels.parse_file(filename))[:nw][0]; kwargs...
)

function getstart(set, item_key, value_key, default = 0.0)
    return get(get(set, item_key, Dict()), value_key, default)
end

function admittancematrix(ref::Dict{Symbol,Any}, bus_index::Dict{Int,Int})
    nbus = length(ref[:bus])
    B = zeros(nbus,nbus)
    for (i,branch) in ref[:branch]
        f_bus = bus_index[ref[:bus][branch["f_bus"]]["index"]]
        t_bus = bus_index[ref[:bus][branch["t_bus"]]["index"]]
        B[f_bus, t_bus] += (-branch["br_x"]/(branch["br_x"]^2+branch["br_r"]^2))
        B[t_bus, f_bus] += (-branch["br_x"]/(branch["br_x"]^2+branch["br_r"]^2))
        B[f_bus, f_bus] += (branch["br_x"]/(branch["br_x"]^2+branch["br_r"]^2))
        B[t_bus, t_bus] += (branch["br_x"]/(branch["br_x"]^2+branch["br_r"]^2))
    end
    B
end

θ(ref, busvalue::Function, i) = sum(ref.π[i,j]*busvalue(j) for j in 1:ref.nbus)
θ(ref, busvalue::Vector, i) = sum(ref.π[i,j]*busvalue[j] for j in 1:ref.nbus)
θ(ref, busvalue::Vector{JuMP.AffExpr}, i) = 
    sum(ref.π[i, j] * busvalue[j] for j in 1:ref.nbus)
# COST FUNCTION
cost(ref::NetworkReference, p::Vector) = sum(
    ref.gen[i].cost[1]*p[i] + ref.gen[i].cost[2]*p[i] + ref.gen[i].cost[3]
    for i in 1:ref.ngen
)
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
function lineflow(ref, p, ω, l)
    function busvalue(i)
        result = ω[i] - ref.bus[i].pd - ref.bus[i].gs
        if !isempty(ref.bus[i].gens)
            result += sum(p[g] for g in ref.bus[i].gens)
        end
        result
    end
    ref.line[l].β*(θ(ref,busvalue,ref.line[l].frombus) - θ(ref,busvalue,ref.line[l].tobus))
end

function lineflow(ref, p, α, ω, l)
    function busvalue(i)
        result = ω[i] - ref.bus[i].pd - ref.bus[i].gs
        if !isempty(ref.bus[i].gens)
            result += sum(p[g] - α[g]*sum(ω) for g in ref.bus[i].gens)
        end
        result
    end
    ref.line[l].β*(θ(ref,busvalue,ref.line[l].frombus) - θ(ref,busvalue,ref.line[l].tobus))    
end

function fulllineflow(ref, p, α, ω, l)
    function busvalue(i)
        result = ω[i] - ref.bus[i].pd - ref.bus[i].gs
        if !isempty(ref.bus[i].gens)
            result += sum(
                p[g] - sum(α[g,j]*ω[j] for j in 1:ref.nbus)
                for g in ref.bus[i].gens
            )
        end
        result
    end
    ref.line[l].β*(θ(ref,busvalue,ref.line[l].frombus) - θ(ref,busvalue,ref.line[l].tobus))
end
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

function SingleScenarioOPF(ref::NetworkReference) # ,solver::MathProgBase.AbstractMathProgSolver
    model = JuMP.Model(Clp.Optimizer)
    # Save ω as a field of the model structure, or as an external variable
    # ω = @variable(model, [i in 1:ref.nbus])
    JuMP.@variable(model,                    ω[i in 1:ref.nbus]) # OLD definition of ω
    JuMP.@variable(model, ref.gen[i].pmin <= p[i in 1:ref.ngen] <= ref.gen[i].pmax, start=ref.gen[i].pstart)
    JuMP.@expression(model, busvalue[i in 1:ref.nbus],
        sum(p[g] for g in ref.bus[i].gens) + ω[i] - ref.bus[i].pd - ref.bus[i].gs 
    )

    lineflow(l) = ref.line[l].β*(
        θ(ref,busvalue,ref.line[l].frombus) - θ(ref,busvalue,ref.line[l].tobus)
    )
    JuMP.@constraints model begin
        [l in 1:ref.nline], lineflow(l) <= ref.line[l].rate
        [l in 1:ref.nline], lineflow(l) >= -ref.line[l].rate
        0 == sum(sum(p[g] for g in ref.bus[i].gens) + ω[i] - ref.bus[i].pd - ref.bus[i].gs
                 for i in 1:ref.nbus)
    end
    JuMP.@objective(model, Min, cost(ref, p))
    # SingleScenarioOPF(model,p,ω)
    return model, p, ω
    # return SingleScenarioOPF(model,p,ω)
end

SingleScenarioOPF(filename::String; kwargs...) =
    SingleScenarioOPF(PM.build_ref(PM.parse_file(filename)); kwargs...)

SingleScenarioOPF(ref::Dict{Symbol,Any}; kwargs...) =
    SingleScenarioOPF(NetworkReference(ref); kwargs...)

SingleScenarioOPF

In [3]:
# DONT' DELETE. CELL DEDICATED FOR COMPLETING THE LOADED DATA FIELD. the loaded cost is correct, × 100. the loaded pd is correct, /100. gs are same since all 0 
data_file = "nesta_case30_ieee.m"
data_matlab = DataFrame(XLSX.readtable("coba.xlsx", data_file*"_B")) # Access manually copied data from .m file to xlxs file
wk = data_matlab[:, [:bus_i, :Pd, :Gs]]
# Adding columns: pd and gs
ref_temp = PowerModels.build_ref(PowerModels.parse_file(data_file))[:it][:pm][:nw][0]
for i in 1:length(ref_temp[:bus])
    ref_temp[:bus][i]["pd"] = wk.Pd[i]/100 # the indexing is correct, the problem is "pd" in the original .m file is 100x larger than the imported powermodels, therefore it's divided by 100
    ref_temp[:bus][i]["gs"] = wk.Gs[i] # from 4 dataset, all Gs's are 0, so no need to do any operation
end
# obtain 3 cost columns
data_matlab = DataFrame(XLSX.readtable("coba.xlsx", data_file*"_C", header=false)) # Access manually copied data from .m file to xlxs file
wk = data_matlab[:, 5:7] # the 3 cost columns
list_of_lists = [[collect(wk[i, :])] for i in 1:size(wk, 1)] # convert each row into list
float_list = [[Float64(list_of_lists[y][1][x]) for x in 1:3] for y in 1:length(list_of_lists)]
# Adding 3 columns of cost 
for i in 1:length(ref_temp[:gen])
    ref_temp[:gen][i]["cost"] = float_list[i]*100 # multiplied by 100 to conform with the output of PowerModels.parse_file(data_file)
end

ref = NetworkReference(ref_temp, bus_prob = 0.95, line_prob = 0.95, σscaling = 0.03);

[info | PowerModels]: removing 3 cost terms from generator 4: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 1: [52.1378, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 5: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 2: [113.51659999999998, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 6: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 3: Float64[]


In [9]:
function get_opf_solution(opf::SingleScenarioOPF, ω)
    for i in eachindex(ω); JuMP.fix(opf.ω[i], ω[i]) end
    @assert JuMP.solve(opf.model) == :Optimal
    JuMP.getvalue(opf.p)
end

get_opf_solution (generic function with 1 method)

In [4]:
m, p, ω = SingleScenarioOPF(ref)  # This now returns the JuMP model, and the variables p and ω
@time optimize!(m)                # Solve the optimization problem

  3.165152 seconds (1.06 M allocations: 72.322 MiB, 2.17% gc time, 98.90% compilation time: 5% of which was recompilation)
Coin0506I Presolve 76 (-7) rows, 30 (-6) columns and 2112 (-532) elements
Clp0006I 0  Obj 0 Primal inf 58.784537 (12)
Clp0006I 5  Obj -1.1629741e-17
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 5 iterations time 0.032, Presolve 0.03


In [5]:
# After solving, you can retrieve the optimized values for p and ω
for i in 1:ref.ngen
    println("p[$i] = ", JuMP.value(p[i]))
end
for i in 1:ref.nbus
    println("ω[$i] = ", JuMP.value(ω[i]))
end

p[1] = 0.0
p[2] = 0.0
p[3] = 0.0
p[4] = 0.0
p[5] = 0.0
p[6] = 0.0
ω[1] = 1.816993371003536
ω[2] = 0.1305449797975759
ω[3] = 0.0
ω[4] = 0.0
ω[5] = 0.0
ω[6] = 0.0
ω[7] = 0.0
ω[8] = 0.0
ω[9] = 0.0
ω[10] = 0.0
ω[11] = 1.419999999999999
ω[12] = 0.0
ω[13] = -0.7924342880228278
ω[14] = 0.0
ω[15] = 0.0
ω[16] = 0.0
ω[17] = 0.08195883292100452
ω[18] = 0.0
ω[19] = 0.005671148769506179
ω[20] = 0.0
ω[21] = 0.0
ω[22] = 0.0
ω[23] = 0.0
ω[24] = 0.0
ω[25] = 0.0
ω[26] = -0.13762868440301265
ω[27] = 0.30889463993421906
ω[28] = 0.0
ω[29] = 0.0
ω[30] = 0.0


In [6]:
# nsamples = 10
# nonzeroindices = (1:length(ref.stdω))[ref.stdω .> 1e-5]
# ω = Distributions.MvNormal(
#     zeros(length(nonzeroindices)),
#     diagm(ref.stdω[nonzeroindices])^2
# )
# ωsamples = zeros(ref.nbus, nsamples)
# ωsamples[nonzeroindices, :] = rand(ω, nsamples)
# nsamples = size(ωsamples, 2)
# status = Array{MOI.TerminationStatusCode}(undef, nsamples)
# soln_p = zeros(nsamples, ref.ngen);
# cbases = Dict{Vector{Symbol},Vector{Int}}()
# rbases = Dict{Vector{Symbol},Vector{Int}}()
# noptimal = 0
# notoptimal = 0

# for i in 1:nsamples
#     for j in ω # error di eachindex(ω)
#         println(j)
# #         # JuMP.fix(ω[j], ωsamples[j,i]) 
#     end
# # #     println("a")
# # #     # JuMP.optimize!(m) # optimize the model (NEW LINE)
# # #     # status[i] = JuMP.termination_status(m) 
# end

In [29]:
# DON'T DELETE THIS CELL
mutable struct OPFScenarios
    ref::NetworkReference
    scenarios::Matrix{Float64} # nscenarios x nuncertain
    solutions::Matrix{Float64} # nscenarios x ngens
    # cbases::Vector{Vector{Symbol}} # nbasis x num_vars
    # rbases::Vector{Vector{Symbol}} # nbasis x num_constrs
    # whichbasis::Matrix{Int} # nscenarios x 2 (indicating index of cbasis and rbasis)
    # whichscenario::Dict{Tuple{Int,Int},Vector{Int}}
end

"""
Takes in `ωsamples` as realizations of the uncertainty
"""
function OPFScenarios(
        ref::NetworkReference,
        # m::SingleScenarioOPF,
        m::JuMP.Model,
        ωsamples::Matrix{Float64}; # nuncertain x nscenarios
    )
    nsamples = size(ωsamples, 2)
    # status = Array{Symbol}(nsamples); original
    status = Array{MOI.TerminationStatusCode}(undef, nsamples)
    soln_p = zeros(nsamples, ref.ngen);
    cbases = Dict{Vector{Symbol},Vector{Int}}()
    rbases = Dict{Vector{Symbol},Vector{Int}}()
    noptimal = 0
    notoptimal = 0

    # p = Progress(nsamples, 1)
    for i in 1:nsamples
        for j in eachindex(ω)
            JuMP.fix(ω[j], ωsamples[j,i]) # gimana caranya print nilai ω[j]
        end
        # JuMP.getvalue(m.ω) #println(ω[j])
        # status[i] = JuMP.optimize!(m) # DON'T DELETE THIS LINE. here's the problem since typeof(JuMP.optimize!(m)) is `nothing`.
        JuMP.optimize!(m) # optimize the model (NEW LINE)
        status[i] = JuMP.termination_status(m) # retrieve and store the status (NEW LINE)
        
        if status[i] == :Optimal
        #     # if the scenario was feasible, keep track of basis
            soln_p[i,:] = JuMP.getvalue(m.p)
        #     noptimal += 1
        #     # cbasis, rbasis = MathProgBase.getbasis(m.model.internalModel)            
        #     # cbasis, rbasis = MOI.get(m, MOI.VariablePrimalBasisStatus(), var)
        #     # cbases[cbasis] = get(cbases, cbasis, Int[])
        #     # rbases[rbasis] = get(rbases, rbasis, Int[])
        #     # push!(cbases[cbasis], noptimal)
        #     # push!(rbases[rbasis], noptimal)
        # else
        #     notoptimal += 1
        end
        # next!(p)
    end
    @assert noptimal == sum(status .== :Optimal)

    sample_p = soln_p[status .== :Optimal, :]
    sample_ω = ωsamples[:,status .== :Optimal]

    # colbases = map(collect,keys(cbases))
    # rowbases = map(collect,keys(rbases))
    # whichcol = Dict(zip(colbases,1:length(colbases)))
    # whichrow = Dict(zip(rowbases,1:length(rowbases)))
    # whichbasis = zeros(Int, noptimal, 2)
    # for k in keys(cbases); whichbasis[cbases[k],1] = whichcol[collect(k)] end
    # for k in keys(rbases); whichbasis[rbases[k],2] = whichrow[collect(k)] end

    # whichscenario = Dict{Tuple{Int,Int},Vector{Int}}()
    # for i in 1:noptimal
    #     basiskey = (whichbasis[i,1], whichbasis[i,2])
    #     whichscenario[basiskey] = get(whichscenario, basiskey, Int[])
    #     push!(whichscenario[basiskey], i)
    # end

    OPFScenarios(ref, sample_ω, sample_p), noptimal, notoptimal
    # OPFScenarios(ref, sample_ω, sample_p, colbases, rowbases, whichbasis, whichscenario)
end


"""
Defaults to MvNormal for the uncertainty and generates nsamples wiht mean 0,
and standard deviation given by ref.stdω
"""

function OPFScenarios(
        ref::NetworkReference,
        # m::SingleScenarioOPF;
        m::JuMP.Model;
        nsamples::Int=1000
    )
    nonzeroindices = (1:length(ref.stdω))[ref.stdω .> 1e-5]
    ω = Distributions.MvNormal(
        zeros(length(nonzeroindices)),
        diagm(ref.stdω[nonzeroindices])^2
    )
    ωsamples = zeros(ref.nbus, nsamples)
    ωsamples[nonzeroindices, :] = rand(ω, nsamples)
    OPFScenarios(ref, m, ωsamples)
end

OPFScenarios

In [56]:
@time scenarios, noptimal, notoptimal = OPFScenarios(ref, m, nsamples = 1);

  0.001210 seconds (377 allocations: 44.391 KiB)
Coin0506I Presolve 0 (-83) rows, 0 (-36) columns and 0 (-2644) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 201.57819
Coin0511I After Postsolve, objective 201.57819, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 201.5781948 - 0 iterations time 0.002, Presolve 0.00


In [16]:
# MOI.get(optimizer, MOI.TerminationStatus())
# MOI.get(optimizer, MOI.ResultCount())
# MOI.get(optimizer, MOI.PrimalStatus())
# MOI.get(optimizer, MOI.DualStatus())
# MOI.get(optimizer, MOI.ObjectiveValue())
# MOI.get(optimizer, MOI.VariablePrimal(), x)